In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from src import RGBModule, RGBDataModule, RGBNirDataModule, RGBNirModule
import pytorch_lightning as pl

dm = RGBNirDataModule(batch_size=512, num_workers=4, pin_memory=True)
dm.setup()

train: 1587395
val: 40080
test: 36421


In [4]:
config = {
    'backbone': 'resnet34',
    'pretrained': True,
    'optimizer': 'Adam',
    'optimizer_params': {
        'lr': 1e-3
    },
    'early_stopping': False,
    'trainer': {
        'gpus': 1,
        'max_epochs': 3,
        'logger': None,
        'enable_checkpointing': False,
        'overfit_batches': 0,
        'deterministic': True,
        'precision': 16
    },
    'datamodule': {
        'batch_size': 780,
        'num_workers': 8,
        'pin_memory': True
    },
}

if __name__ == "__main__":
    model = RGBNirModule(config)
    trainer = pl.Trainer(**config["trainer"], resume_from_checkpoint="checkpoints/r34rgbnir_da-val_loss=6.54345-epoch=3.ckpt")
    #model = RGBModule.load_from_checkpoint("checkpoints/baseline-val_loss=8.22737-epoch=0.ckpt")
    trainer.fit(model, dm)
    model.hparams

Using 16bit native Automatic Mixed Precision (AMP)
d:\Code\Kaggle_Competitions\GeoLifeCLEF2022\venv\lib\site-packages\pytorch_lightning\trainer\connectors\checkpoint_connector.py:51: LightningDeprecationWarning: Setting `Trainer(resume_from_checkpoint=)` is deprecated in v1.5 and will be removed in v1.7. Please pass `Trainer.fit(ckpt_path=)` directly instead.
  rank_zero_deprecation(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
d:\Code\Kaggle_Competitions\GeoLifeCLEF2022\venv\lib\site-packages\pytorch_lightning\trainer\trainer.py:805: LightningDeprecationWarning: `trainer.resume_from_checkpoint` is deprecated in v1.5 and will be removed in v2.0. Specify the fit checkpoint path with `trainer.fit(ckpt_path=)` instead.
  ckpt_path = ckpt_path or self.resume_from_checkpoint
Restoring states from the checkpoint path at checkpoints/r34rgbnir_da-val_loss=6.54345-epoch=3.ckpt


train: 1587395
val: 40080
test: 36421


d:\Code\Kaggle_Competitions\GeoLifeCLEF2022\venv\lib\site-packages\pytorch_lightning\trainer\trainer.py:1718: UserWarning: Be aware that when using `ckpt_path`, callbacks used to create the checkpoint need to be provided during `Trainer` instantiation. Please add the following callbacks: ["ModelCheckpoint{'monitor': 'val_loss', 'mode': 'min', 'every_n_train_steps': 0, 'every_n_epochs': 1, 'train_time_interval': None, 'save_on_train_epoch_end': True}"].
  rank_zero_warn(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type   | Params
---------------------------------
0 | model | ResNet | 30.0 M
---------------------------------
30.0 M    Trainable params
0         Non-trainable params
30.0 M    Total params
60.056    Total estimated model params size (MB)
Restored all states from the checkpoint file at checkpoints/r34rgbnir_da-val_loss=6.54345-epoch=3.ckpt


In [5]:
from tqdm import tqdm

model.cuda()
dl = dm.test_dataloader()
labels, observations = [], []
for imgs, observation_ids in tqdm(dl):
    preds = model.predict(imgs)
    values, ixs = preds.topk(30)
    labels += [' '.join([str(i.item()) for i in ix]) for ix in ixs]
    observations += observation_ids
    

  8%|▊         | 6/72 [00:20<01:46,  1.62s/it]Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x000001D37997DFC0>
Traceback (most recent call last):
  File "d:\Code\Kaggle_Competitions\GeoLifeCLEF2022\venv\lib\site-packages\torch\utils\data\dataloader.py", line 1358, in __del__
    self._shutdown_workers()
  File "d:\Code\Kaggle_Competitions\GeoLifeCLEF2022\venv\lib\site-packages\torch\utils\data\dataloader.py", line 1289, in _shutdown_workers
    if not self._shutdown:
AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute '_shutdown'
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x000001D37997DFC0>
Traceback (most recent call last):
  File "d:\Code\Kaggle_Competitions\GeoLifeCLEF2022\venv\lib\site-packages\torch\utils\data\dataloader.py", line 1358, in __del__
    self._shutdown_workers()
  File "d:\Code\Kaggle_Competitions\GeoLifeCLEF2022\venv\lib\site-packages\torch\utils\data\dataloader.py", line 1289, in _shu

In [6]:
import pandas as pd
submission = pd.DataFrame({"Id": int(observations), "Predicted": labels})
submission.to_csv("submission.csv", index=False)
submission.sample(5)

,Id,Predicted
32128,tensor(21606530),3962 5227 5686 5045 5192 5504 3072 6739 5168 5...
5687,tensor(10709993),191 118 743 1059 223 515 131 416 205 209 30 56...
22835,tensor(20601479),6028 1432 2044 5796 5794 7413 4903 1767 4870 1...
22589,tensor(20573605),5311 5058 5409 5068 6350 12203 7897 12181 5322...
21320,tensor(20440587),1227 2682 2561 1075 2386 1792 5364 774 2334 50...


In [7]:
submission["Id"] = submission["Id"].astype(int)

In [8]:
submission.to_csv("submission.csv", index=False)
submission.sample(5)


,Id,Predicted
20145,20312324,1597 5356 6184 1452 2200 2268 5901 6818 2674 2...
28101,21175631,2723 1092 5666 5940 2329 5194 5 5688 6171 6049...
15761,10030437,944 29 188 327 527 908 409 371 748 511 426 751...
28281,21196626,5612 5504 5633 5045 5402 5011 5969 5698 5366 4...
30640,21446244,5020 5068 4982 5668 5531 5359 3953 5298 4988 5...


In [9]:
sample_submission = pd.read_csv("../data/sample_submission.csv")
assert len(sample_submission) == len(submission)
sample_submission.sample(10)

,Id,Predicted
27021,21058535,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
34204,21825709,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
26623,21016851,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
11934,10571438,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
18462,20124927,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
26604,21014972,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
7718,10366128,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
6828,10321425,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
2377,10113023,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
12097,10580254,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
